# 01. Библиотека PyTorch

## План
1. `numpy` -> `pytorch`
2. Автоматическое дифференцирование (`torch.autograd`)

Если успеем

3. Пример: задача регрессии
4. BONUS! Переопределение `backward()`


[PyTorch](https://pytorch.org) - это фреймворк для машинного обучения:
* В первую очередь - для нейросетевых моделей
* Большие возможности для работы с тензорами
* Поддержка выполнения на CPU / GPU / TPU ([!](https://github.com/pytorch/xla))
* Автоматическое дифференцирование вычислительных графов (`torch.autograd`)
* Широкий набор строительных блоков для DL-моделей (`torch.nn`) и готовых архитектур (`torchvision.models`+)
* Удобная поддержка реализации своих операций / слоев / функций потерь / ...

## 1. `numpy` <-> `pytorch`

In [5]:
import torch
import numpy as np
import time

### 1.1. Работа с тензорами

~Все, что можно делать с многомерными массивами в `numpy`, можно делать и в `pytorch`:
* `np.ndarray` -> `torch.Tensor`
    * [broadcasting](https://numpy.org/doc/stable/user/basics.broadcasting.html)
* `np.zeros()` -> `torch.zeros()`
* `np.stack()` -> `torch.stack()`
* `np.concatenate` -> `torch.cat()` 
* `np.random.normal()` -> `torch.randn()`
* ...

#### `np.ndarray` <-> `torch.Tensor`

In [6]:
data_np = np.random.uniform(size=(3, 3))
data_np.shape

(3, 3)

In [7]:
data_np

array([[0.61057018, 0.20730497, 0.42520123],
       [0.36197064, 0.83858856, 0.47160997],
       [0.10333268, 0.71592089, 0.51099634]])

Создание тензора из данных в форме `numpy.ndarray` через [`torch.from_numpy()`](https://pytorch.org/docs/stable/generated/torch.from_numpy.html):

In [8]:
data_pt = torch.from_numpy(data_np)
data_pt.shape

torch.Size([3, 3])

In [9]:
data_np == data_pt.numpy()  # <- get data from Tensor as numpy array

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])

In [10]:
data_np.dtype

dtype('float64')

In [11]:
data_pt.dtype

torch.float64

Важно: при создании тензора через `torch.from_numpy()` данные шарятся между объектами:

In [12]:
t = data_pt.numpy() 
t -= 1000
data_pt

tensor([[-999.3894, -999.7927, -999.5748],
        [-999.6380, -999.1614, -999.5284],
        [-999.8967, -999.2841, -999.4890]], dtype=torch.float64)

In [13]:
data_np

array([[-999.38942982, -999.79269503, -999.57479877],
       [-999.63802936, -999.16141144, -999.52839003],
       [-999.89666732, -999.28407911, -999.48900366]])

Создание тензора через конструктор `torch.tensor()`:

In [14]:
data_np = np.random.uniform(size=(3, 3))
data_np.shape

(3, 3)

In [15]:
data_pt = torch.tensor(data_np)
data_pt.shape

torch.Size([3, 3])

In [16]:
data_np == data_pt.numpy()

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])

При создании тензора через `torch.tensor()` данные копируются:

In [17]:
t = data_pt.numpy()
t -= 1000
data_pt

tensor([[-999.2775, -999.6246, -999.9425],
        [-999.8026, -999.9138, -999.6672],
        [-999.6515, -999.1996, -999.2476]], dtype=torch.float64)

In [18]:
data_np

array([[0.72254241, 0.37536442, 0.05751275],
       [0.19735192, 0.08617383, 0.33283387],
       [0.34847346, 0.8004295 , 0.75235173]])

**NB**: [`torch.Tensor()`](https://glaringlee.github.io/tensors.html?highlight=torch%20tensor#torch.Tensor) != [`torch.tensor()`](https://glaringlee.github.io/generated/torch.tensor.html?highlight=torch%20tensor#torch.tensor)!

#### Типы данных

In [19]:
data_np.dtype

dtype('float64')

floats

In [20]:
data_pt = torch.tensor(data_np)
data_pt.dtype

torch.float64

In [21]:
data_pt = torch.tensor(data_np, dtype=torch.float)
data_pt.dtype

torch.float32

In [25]:
data_pt = torch.tensor(data_np, dtype=torch.float16)
data_pt.dtype

torch.float16

In [23]:
data_pt = torch.tensor(data_np, dtype=torch.half)
data_pt.dtype


torch.float16

In [34]:
data_pt = torch.tensor(data_np, dtype=torch.double)
data_pt.dtype

torch.float64

ints

In [30]:
data_pt = torch.tensor(data_np, dtype=torch.int)
data_pt.dtype

torch.int32

In [31]:
data_pt = torch.tensor(data_np, dtype=torch.long)
data_pt.dtype

torch.int64

In [33]:
data_pt

tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]], dtype=torch.int8)

In [32]:
data_pt = torch.tensor(data_np, dtype=torch.int8)
data_pt.dtype

torch.int8

Больше - [тут](https://pytorch.org/docs/stable/tensors.html).

Можно и без `numpy`, разумеется:

In [50]:
torch.tensor(2)

tensor(2)

In [47]:
torch.tensor([1, 2, 3, 4, 5]).size()

torch.Size([5])

In [51]:
torch.tensor([[0., 1], [2, 3]])

tensor([[0., 1.],
        [2., 3.]])

Если в тензоре один-единственный элемент, извлечь его численное значение можно через `.item()`:

In [55]:
torch.tensor(10).item()

10

In [58]:
all_losses = []
x = torch.tensor(10)
all_losses.append(x.item())
#x.item()

In [59]:
all_losses

[10]

In [65]:
x = torch.tensor([10, 11])
x.numpy()

array([10, 11])

**NB**: поведение конструкторов `torch.tensor()` и `torch.Tensor()` - разное:

In [66]:
torch.Tensor([1, 2, 3])

tensor([1., 2., 3.])

In [71]:
torch.Tensor(1)

tensor([2.8676e-35])

In [74]:
torch.Tensor(2)

tensor([2.1359e-35, 0.0000e+00])

#### Broadcasting

In [75]:
data_np = np.random.uniform(size=(3, 3))
data_pt = torch.from_numpy(data_np)
data_pt

tensor([[0.6317, 0.3461, 0.8336],
        [0.7828, 0.6741, 0.0319],
        [0.1103, 0.9823, 0.4292]], dtype=torch.float64)

In [76]:
data_pt -= 100
data_pt

tensor([[-99.3683, -99.6539, -99.1664],
        [-99.2172, -99.3259, -99.9681],
        [-99.8897, -99.0177, -99.5708]], dtype=torch.float64)

Обратите внимание на особенности broadcasting в "неочевидных" случаях (часто лучше перепроверить):

In [77]:
data_np = np.random.uniform(size=(3, 3)).astype(np.float32)
data_pt = torch.from_numpy(data_np)
data_pt

tensor([[0.3676, 0.9412, 0.0133],
        [0.3400, 0.2429, 0.2695],
        [0.4998, 0.2473, 0.7413]])

In [78]:
b = torch.Tensor([100, 50, 0])
b.shape

torch.Size([3])

In [79]:
b

tensor([100.,  50.,   0.])

In [80]:
data_pt + b

tensor([[1.0037e+02, 5.0941e+01, 1.3272e-02],
        [1.0034e+02, 5.0243e+01, 2.6946e-01],
        [1.0050e+02, 5.0247e+01, 7.4125e-01]])

In [81]:
b = torch.Tensor([100, 50, 0]).view(1, 3)
b.shape

torch.Size([1, 3])

In [82]:
data_pt + b

tensor([[1.0037e+02, 5.0941e+01, 1.3272e-02],
        [1.0034e+02, 5.0243e+01, 2.6946e-01],
        [1.0050e+02, 5.0247e+01, 7.4125e-01]])

In [83]:
b = torch.Tensor([100, 50, 0]).view(3, 1)
b.shape

torch.Size([3, 1])

In [84]:
data_pt + b

tensor([[100.3676, 100.9412, 100.0133],
        [ 50.3400,  50.2429,  50.2695],
        [  0.4998,   0.2473,   0.7413]])

In [93]:
data_pt[0] + b.squeeze(1)

tensor([100.3400,  50.2429,   0.2695])

#### Векторные и матричные операции

In [94]:
data_np = np.random.uniform(size=(4, 4))
data_pt = torch.from_numpy(data_np)

In [95]:
data_np @ data_np

array([[2.24230663, 1.7863323 , 1.20140162, 1.79099281],
       [1.03659108, 0.80515092, 0.61922261, 0.80112764],
       [1.86583742, 1.44512777, 1.0584246 , 1.4799741 ],
       [1.92782077, 1.55864462, 1.05478925, 1.58620214]])

In [96]:
data_pt @ data_pt

tensor([[2.2423, 1.7863, 1.2014, 1.7910],
        [1.0366, 0.8052, 0.6192, 0.8011],
        [1.8658, 1.4451, 1.0584, 1.4800],
        [1.9278, 1.5586, 1.0548, 1.5862]], dtype=torch.float64)

In [97]:
torch.matmul(data_pt, data_pt)

tensor([[2.2423, 1.7863, 1.2014, 1.7910],
        [1.0366, 0.8052, 0.6192, 0.8011],
        [1.8658, 1.4451, 1.0584, 1.4800],
        [1.9278, 1.5586, 1.0548, 1.5862]], dtype=torch.float64)

In [98]:
x = torch.randn(4, 1)
A = torch.randn(8, 4)

Умножение матрицы на вектор:

In [99]:
y = A @ x
y.shape, y

(torch.Size([8, 1]),
 tensor([[ 0.2815],
         [ 0.3679],
         [ 1.6850],
         [-2.6279],
         [-1.1225],
         [-3.0399],
         [ 1.8857],
         [ 2.9293]]))

In [100]:
y = A.matmul(x)
y.shape, y

(torch.Size([8, 1]),
 tensor([[ 0.2815],
         [ 0.3679],
         [ 1.6850],
         [-2.6279],
         [-1.1225],
         [-3.0399],
         [ 1.8857],
         [ 2.9293]]))

Тоже повнимательнее с размерностями:

In [104]:
x = torch.randn(4)
A = torch.randn(8, 4)

In [105]:
y = A @ x
y.shape, y

(torch.Size([8]),
 tensor([ 0.3699,  2.1246, -1.1579, -0.0652,  1.5893, -1.0164,  0.3715,  1.3674]))

#### Задание:

* Даны два набора векторов одинаковой размерности, `X` (`m x dim`) и `Y` (`n x dim`)
* Требуется посчитать (без циклов!) попарные значения косинуса углов для всех пар (`x`, `y`)
    * На выходе ожидается матрица размером (`m x n`)

In [113]:
dim = 8
m = 11
n = 17
X = torch.randn(m, dim)
Y = torch.randn(n, dim)

# YOUR CODE HERE
X_norm = (X@X.T).diag().sqrt()
Y_norm = (Y@Y.T).diag().sqrt()

pairiwse_norms = X_norm.unsqueeze(1) @ Y_norm.unsqueeze(1).T

cos_mat = X@Y.T / pairiwse_norms
# END OF YOUR CODE

In [114]:
from scipy.spatial.distance import cdist
cos_mat_expected = 1 - cdist(X, Y, metric="cosine")

torch.testing.assert_allclose(cos_mat, cos_mat_expected)

/home/victor/anaconda3/lib/python3.7/site-packages/torch/testing/_deprecated.py:35: FutureWarning: torch.testing.assert_allclose() is deprecated since 1.12 and will be removed in 1.14. Use torch.testing.assert_close() instead. For detailed upgrade instructions see https://github.com/pytorch/pytorch/issues/61844.
  warnings.warn(msg, FutureWarning)


#### Работа с размерностями

In [116]:
x = torch.randn(3, 1)
x.shape, x

(torch.Size([3, 1]),
 tensor([[ 0.5233],
         [-0.0374],
         [ 1.0064]]))

Транспонирование:

In [117]:
xT = x.T
xT.shape, xT

(torch.Size([1, 3]), tensor([[ 0.5233, -0.0374,  1.0064]]))

Если не указано явно, то данные НЕ копируются:

In [118]:
xT *= 100
xT

tensor([[ 52.3254,  -3.7446, 100.6448]])

In [119]:
x

tensor([[ 52.3254],
        [ -3.7446],
        [100.6448]])

С копированием:

In [124]:
xT_clone = x.T.clone()
xT_clone.shape, xT_clone

(torch.Size([1, 3]), tensor([[ 52.3254,  -3.7446, 100.6448]]))

In [125]:
xT_clone == xT

tensor([[True, True, True]])

In [126]:
xT_clone *= 100
xT_clone

tensor([[ 5232.5420,  -374.4568, 10064.4766]])

In [130]:
x

tensor([[ 52.3254],
        [ -3.7446],
        [100.6448]])

In [136]:
xT_clone = torch.tensor(x)

/home/victor/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.


In [138]:
xT_clone *= 0

In [139]:
x

tensor([[ 52.3254],
        [ -3.7446],
        [100.6448]])

Доступны и более сложные операции над размерностями.

`permute()`:

In [140]:
x = torch.randn(3, 64, 256)
x.shape

torch.Size([3, 64, 256])

In [141]:
x.permute(1, 2, 0).shape

torch.Size([64, 256, 3])

In [142]:
x.permute(1, 2, 0)[:,:,0] == x[0,:,:]

tensor([[True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        ...,
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True]])

In [109]:
rnn_out = torch.randn(32, 64, 128 )

In [111]:
rnn_out.permute(0,2,1).size()

torch.Size([32, 128, 64])

In [105]:
x.transpose(1, 2).shape

torch.Size([3, 256, 64])

`view()`:

In [155]:
x = torch.randn(3, 64, 256)
x.shape

torch.Size([3, 64, 256])

In [144]:
x.view(3, 256, 64).shape

torch.Size([3, 256, 64])

In [114]:
x.view(3, -1).shape

torch.Size([3, 16384])

In [150]:
x.view(3, 256, -1).size()

torch.Size([3, 256, 64])

In [156]:
z = x.view(-1)
z

tensor([ 1.7303,  0.3510,  0.0948,  ..., -0.0750, -0.7927, -1.0080])

In [157]:
z *= 0

In [158]:
z

tensor([0., 0., 0.,  ..., -0., -0., -0.])

In [159]:
x

tensor([[[0., 0., 0.,  ..., -0., -0., 0.],
         [-0., 0., 0.,  ..., -0., 0., 0.],
         [-0., -0., 0.,  ..., -0., 0., -0.],
         ...,
         [-0., -0., -0.,  ..., 0., -0., 0.],
         [-0., 0., 0.,  ..., 0., 0., 0.],
         [-0., -0., -0.,  ..., 0., -0., 0.]],

        [[0., 0., -0.,  ..., 0., -0., -0.],
         [0., 0., 0.,  ..., -0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., -0.],
         ...,
         [-0., 0., -0.,  ..., -0., 0., -0.],
         [-0., -0., -0.,  ..., 0., 0., -0.],
         [0., -0., 0.,  ..., 0., -0., -0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [-0., -0., 0.,  ..., -0., 0., -0.],
         [-0., -0., 0.,  ..., -0., 0., -0.],
         ...,
         [0., 0., 0.,  ..., -0., -0., -0.],
         [0., 0., 0.,  ..., -0., 0., -0.],
         [0., 0., 0.,  ..., -0., -0., -0.]]])

Больше - [тут](https://jhui.github.io/2018/02/09/PyTorch-Basic-operations/).

#### Задание:

* Создать тензор размером `1x3x224x224` (имитируем батч из одной картинки с 3 каналами RGB размером `224х224`)
    * Все значения в тензоре = `0.5`
* Выполнить нормализацию через mean / std из датасета Imagenet:
    * mean = `[0.485, 0.456, 0.406]`
    * std = `[0.229, 0.224, 0.225]`
* Посчитать среднее значение по каждому из 3-х каналов полученного тензора
    * Ответ (`output`) должен иметь размерность (3,)


NB: агрегирующие операции типа `min`/`max`/`mean`/... устроены аналогично в `numpy` и `torch`, но отличаютеся ключевым словом для уточнения размерностей:
* в `numpy` - `axis`
* в `torch` - `dim`

In [161]:
X = torch.ones(1,3,224,224) * 0.5

mean = torch.tensor([0.485, 0.456, 0.406])
std = torch.tensor([0.229, 0.224, 0.225])

normalized = (X - mean.view(-1,1,1)) / std.view(-1,1,1)
output = normalized.mean(dim=[2,3]).squeeze(-1)

# END OF YOUR CODE

In [163]:
output = normalized.mean(dim=[2,3])

In [165]:
expected_output = torch.Tensor([0.0655, 0.1964, 0.4178])
torch.testing.assert_allclose(output.squeeze(0), expected_output)

/home/victor/anaconda3/lib/python3.7/site-packages/torch/testing/_deprecated.py:35: FutureWarning: torch.testing.assert_allclose() is deprecated since 1.12 and will be removed in 1.14. Use torch.testing.assert_close() instead. For detailed upgrade instructions see https://github.com/pytorch/pytorch/issues/61844.
  warnings.warn(msg, FutureWarning)


### 1.2. CPU <-> GPU

`CUDA` - это библиотека для вычислений на графических ускорителях (от NVidia).
`torch` умеет работать с GPU, но для этого в системе должны быть установлены совместимые версии `torch` / `cudatoolkit`. 
Подробнее можно почитать на [официальном сайте](https://pytorch.org/get-started/locally/).

#### Получение информации о доступных "девайсах"

In [123]:
torch.cuda.is_available()

False

In [ ]:
torch.cuda.device_count()

In [ ]:
device_alias = "cuda:0"
# device_alias = "cpu"

In [ ]:
torch.cuda.get_device_name(device_alias)

In [ ]:
device = torch.device(device_alias)

In [ ]:
device

#### Перемещение тензоров между девайсами: `.to(...)`

In [ ]:
data_pt = torch.randn(8, 8)
data_pt.device

In [ ]:
data_pt = data_pt.to(device)
data_pt.device

In [ ]:
a = data_pt.to(torch.device("cpu"))
b = data_pt.to("cpu")
c = data_pt.cpu()

a.device, b.device, c.device

#### Скорость вычислений

`8х8` @ `8x8`

In [ ]:
data_pt = torch.randn(8, 8)
data_pt.device

In [ ]:
%timeit data_pt @ data_pt

In [ ]:
data_pt = data_pt.to(device)
data_pt.device

In [ ]:
data_pt @ data_pt;

In [ ]:
%timeit data_pt @ data_pt

`64х64` @ `64x64`

In [ ]:
data_pt = torch.randn(64, 64)
data_pt.device

In [ ]:
%timeit data_pt @ data_pt

In [ ]:
data_pt = data_pt.to(device)
data_pt.device

In [ ]:
%timeit data_pt @ data_pt

`256х256` @ `256x256`

In [ ]:
data_pt = torch.randn(256, 256)
data_pt.device

In [ ]:
%timeit data_pt @ data_pt

In [ ]:
data_pt = data_pt.to(device)
data_pt.device

In [ ]:
%timeit data_pt @ data_pt

#### Объекты с разных девайсов

In [ ]:
data_pt = torch.randn(3, 3)
data_pt = data_pt.to(device)
data_pt.device

In [ ]:
data_pt.numpy()

In [ ]:
data_pt.to(torch.device("cpu")).numpy()

Взаимодействовать друг с другом могут только объекты, лежащие на одном девайсе

In [ ]:
data_pt_1 = torch.randn(3, 3)
data_pt_1 = data_pt_1.to(device)
data_pt_1.device

In [ ]:
data_pt_2 = torch.randn(3, 3)
data_pt_2 = data_pt_2#.to("cpu")
data_pt_2.device

In [ ]:
data_pt_1 + data_pt_2

In [ ]:
data_pt_1 + data_pt_2.to(data_pt_1.device)

In [ ]:
data_pt_2.device

#### Поддержка "особенных" типов на GPU

https://docs.nvidia.com/deeplearning/tensorrt/support-matrix/index.html#hardware-precision-matrix

In [ ]:
data_pt = torch.randn(3, 3).type(torch.float16).to(device)

In [ ]:
data_pt + 1

In [ ]:
data_pt = torch.randn(3, 3).type(torch.int8).to(device)

In [ ]:
data_pt + 1

Прекрасно, что `pytorch` умеет делать все то же, что и `numpy`.
Но зачем он нужен, если *уже есть* `numpy`?

## 2. Автоматическое дифференцирование

Заметим, что алгебраические выражения можно интерпретировать как вычислительные графы:

* $f(x) = w1 \times x + w2 \times y + w3 \rightarrow $

![comp_graph_03](https://i.ibb.co/f8yv1Kc/aim-seminar01-compgraphs-11-1.png)

Чтобы было проще жить, теперь у нас есть `torch`:

#### `.backward()`

In [ ]:
def f(x, y, w1, w2, w3):
    return w1 * x + w2 * y + w3

In [ ]:
x = torch.tensor([1.])
x.requires_grad

In [ ]:
y = torch.tensor([2.])
y.requires_grad

Для "включения" градиентов у переменной, нужно об этом прямо заявить: 

In [ ]:
w1 = torch.tensor([0.33])
w1.requires_grad_(True)
w1.requires_grad

In [ ]:
w2 = torch.tensor([-1.5]).requires_grad_(True)
w2.requires_grad

In [ ]:
w3 = torch.tensor([0.01], requires_grad=True)
w3.requires_grad

In [ ]:
variables = [x, y, w1, w2, w3]
names = ["x", "y", "w1", "w2", "w3"]

In [ ]:
print("name\tval\tgrad")
for n, v in zip(names, variables):
    print(f"{n}\t{v.item():2.3f}\t{v.grad}")

Еще раз насладимся магией автоматического дифференцирования:

In [ ]:
output = f(x, y, w1, w2, w3)
output

In [ ]:
print("name\tval\tgrad")
for n, v in zip(names, variables):
    print(f"{n}\t{v.item():2.3f}\t{v.grad}")

In [ ]:
output.backward()

In [ ]:
print("name\tval\tgrad")
for n, v in zip(names, variables):
    print(f"{n}\t{v.item():2.3f}\t{v.grad}")

Попробуем вызвать `backward()` еще раз:

In [ ]:
output.backward()

**Вопрос:** зачем может быть нужно делать `backward()` больше одного раза?

In [ ]:
del output
for var in variables:
    var.grad = None

In [ ]:
print("name\tval\tgrad")
for n, v in zip(names, variables):
    print(f"{n}\t{v.item():2.3f}\t{v.grad}")

In [ ]:
output = f(x, y, w1, w2, w3)
output

In [ ]:
output.backward(retain_graph=True)

In [ ]:
print("name\tval\tgrad")
for n, v in zip(names, variables):
    print(f"{n}\t{v.item():2.3f}\t{v.grad}")

In [ ]:
output.backward()

In [ ]:
print("name\tval\tgrad")
for n, v in zip(names, variables):
    print(f"{n}\t{v.item():2.3f}\t{v.grad}")

In [ ]:
output.backward()

#### `torch.no_grad()`

Данный контекст гарантирует, что во всех вычислениях внутри него будут отключены градиенты.

In [ ]:
x = torch.randn(64, 512, 16, 16)
x.requires_grad_(True);

In [ ]:
y = (10 * x).sum()

In [ ]:
y.requires_grad

In [ ]:
with torch.no_grad():
    z = (100 * x).sum()

In [ ]:
z.requires_grad

In [ ]:
x.requires_grad

Зачем это может быть нужно?

Посмотрим на потребление памяти при использовании функции активации сигмоида:

In [ ]:
import torch

device = torch.device("cuda:0")

from torch.nn import Sigmoid

In [ ]:
bytes_in_kilobyte = 1024

def get_allocated_memory():
    mem_MB = torch.cuda.memory_allocated(device=device) / bytes_in_kilobyte
    return round(mem_MB, 3)

def get_tensor_memory(t):
    mem_MB = t.nelement() * t.element_size() / bytes_in_kilobyte
    return round(mem_MB, 3)

In [ ]:
torch.cuda.empty_cache()
get_allocated_memory()

In [ ]:
s = Sigmoid().to(device)
get_allocated_memory()

In [ ]:
x = torch.randn(64, 512, 16, 16).to(device)
x.requires_grad_(True);
get_tensor_memory(x)

In [ ]:
64 * 512 * 16 * 16 * 4 / 1024

In [ ]:
get_allocated_memory()

In [ ]:
y = s(x).mean()
get_tensor_memory(y)

In [ ]:
get_allocated_memory()

In [ ]:
y.backward()
get_allocated_memory()

Сделаем то же самое, но с `torch.no_grad()`:

In [ ]:
# torch.cuda.empty_cache()
get_allocated_memory()

In [ ]:
s = Sigmoid().to(device)
get_allocated_memory()

In [ ]:
x = torch.randn(64, 512, 16, 16).to(device)
x.requires_grad_(True);
get_tensor_memory(x)

In [ ]:
get_allocated_memory()

In [ ]:
with torch.no_grad():
    y = s(x).mean()
get_tensor_memory(y)

In [ ]:
get_allocated_memory()

Почему потребление памяти уменьшилось?
Подробнее [тут](https://medium.com/deep-learning-for-protein-design/a-comprehensive-guide-to-memory-usage-in-pytorch-b9b7c78031d3) или [тут](https://pytorch.org/tutorials/recipes/recipes/tuning_guide.html).

## 3. Пример задачи регрессии

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tqdm
import torch

In [ ]:
np.random.seed(0xAB0BA)
_a = np.random.uniform(1, 5)
_b = np.random.uniform(-3, 3)
_c = np.random.uniform(-3, 3)

num_samples = 100

xs = np.random.uniform(-3, 3, size=num_samples)
ys_clean = _a * xs ** 2 + _b * xs + _c
ys_noise = np.random.normal(0, 1, size=len(ys_clean))
ys = ys_clean + ys_noise

plt.figure(figsize=(12, 5))
plt.scatter(xs, ys, label="gt", s=5)
plt.xlabel("x")
plt.ylabel("y")
plt.legend()
plt.grid(True)

In [ ]:
def model(x, a, b, c):
    return a * x ** 2 + b * x + c

In [ ]:
# device = torch.device("cuda:0")
device = torch.device("cpu")

Создадим переменные `xs_device` и `ys_device`, которые будут содержать обучающие данные сразу на нужном девайсе в виде тензоров.

**Вопрос:** `requires_grad=True` или `requires_grad=False`?

In [ ]:
# YOUR CODE HERE

xs_device = ...
ys_device = ...

# END OF YOUR CODE

Теперь инициализируем веса `a`, `b` и `c` нормальным распределением:

In [ ]:
# YOUR CODE HERE

a = ...
b = ...
c = ...

# END OF YOUR CODE

Допишем рутину обучения, чтобы получить значения весов модели.

In [ ]:
num_epochs = 300
lr = 1e-4
indices = np.arange(len(xs))

loss_list = []
a_list = []
b_list = []
c_list = []


for epoch in tqdm.trange(num_epochs):
    np.random.shuffle(indices)

    loss_epoch = []
    for i in indices:
        
        # YOUR CODE HERE
        
        x = ...
        y_true = ...
        y_pred = ... 
        loss = ...
        
        # END OF YOUR CODE
        
        with torch.no_grad():
            a.data -= lr * a.grad.data
            b.data -= lr * b.grad.data
            c.data -= lr * c.grad.data
            a.grad.zero_()
            b.grad.zero_()
            c.grad.zero_()
            
        loss_epoch.append(loss.item())
        
    loss_list.append(np.mean(loss_epoch))
    a_list.append(a.item())
    b_list.append(b.item())
    c_list.append(c.item())

А теперь посмотрим на результаты:

In [ ]:
xs_sorted = np.sort(xs)

ys_pred_per_epoch = []
for epoch in range(num_epochs):
    a_epoch = a_list[epoch]
    b_epoch = b_list[epoch]
    c_epoch = c_list[epoch]
    with torch.no_grad():
        ys_pred_epoch = model(xs_sorted, a_epoch, b_epoch, c_epoch)
    ys_pred_per_epoch.append(ys_pred_epoch)

In [ ]:
plt.figure(figsize=(12, 5))
plt.scatter(xs, ys, label="gt", s=5, c="r")
plt.plot(xs_sorted, ys_pred_per_epoch[-1], label="pred", c="g")
plt.xlabel("x")
plt.ylabel("y")
plt.legend()
plt.grid(True)
plt.show()

Если есть нужные зависимости у `jupyterlab` (**осторожно, может все сломать**), можно сделать интерактивно:

In [ ]:
#!pip install ipympl

In [ ]:
%matplotlib ipympl
from ipywidgets import *
import numpy as np
import matplotlib.pyplot as plt

x = xs_sorted

def f(x, i):
    return model(x, a_list[i], b_list[i], c_list[i])

fig = plt.figure(figsize=(12, 5))
ax = fig.add_subplot(1, 1, 1)
ax.set_xlabel("x")
ax.set_ylabel("y")

ax.scatter(xs, ys, label="gt", s=5, c="r")
line, = ax.plot(x, f(x, i), label="pred", c="g")

ax.legend()
ax.grid(True)


def update(i=0):
    line.set_ydata(f(x, i))
    fig.canvas.draw_idle()
    
interact(update, i=(0,num_epochs-1,1));

In [ ]:
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(loss_list)
plt.xlabel("epoch")
plt.ylabel("loss")
plt.grid(True)
plt.yscale("log")

plt.subplot(1, 2, 2)
for p_name, p_gt, p_list, color in zip(("a", "b", "c"), (_a, _b, _c), (a_list, b_list, c_list), ("r", "g", "b")):
    plt.hlines(y=p_gt, xmin=0, xmax=num_epochs, linestyles="dashed", colors=color)
    plt.scatter(range(len(p_list)), p_list, label=p_name, s=1, c=color)
plt.xlabel("epoch")
plt.ylabel("param value")
plt.legend()
plt.grid(True)

plt.show()

## 4. Переопределение `backward`

Что, если нам хочется релизовать кстомный градиент для произвольной функции. 

Зачем?

 - Мы можем знать лучший способ посчитать градиент, чем делать бэкпроп для суперпозиции элементарных функций
 - Можем реализовать численно более устойчивый метод
 - Можем использовать функции из внешних библиотек
 - Использовать недифференцируемые функции?..

Рассмотрим сигмоиду:

$$ 
  \sigma(x) = \frac{1}{1+e^{-x}}
$$

Если честно распишем суперпозицию функций, то получим:

$$
  \sigma(x) = f_1 \odot f_2  \odot f_3 \odot f_4(x), where 
$$

$$
f_1 = \frac{1}{u}, f_2 = 1 + u, f_3 = \exp(u), f_4 = -u
$$

Тогда:

$$
\frac{\partial \sigma}{\partial x} = \frac{\partial \sigma}{\partial f_2}\frac{\partial f_2}{\partial f_3}
\frac{\partial f_3}{\partial f_4}
\frac{\partial f_4}{\partial x}
$$

Но зная как устроена производная можно упростить:

$$
\frac{\partial \sigma}{\partial x} = \sigma(x)(1 - \sigma(x))
$$

Вручную задать градиени функции в библиотеке PyTorch можно создав дочерний класс от [`torch.autograd.Function`](https://pytorch.org/docs/stable/notes/extending.html#extending-torch-autograd).

**NB**: Для того, чтобы сделать `backward()`, требуется знать результат `forward()`. Для этого у переменной контекста есть метод [`save_for_backward()`](https://pytorch.org/docs/stable/generated/torch.autograd.function.FunctionCtx.save_for_backward.html).

In [ ]:
class MySigmoid(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x):
        # YOUR CODE HERE
        
        val = ...
        
        
        # END OF YOUR CODE
        return val
    
    @staticmethod
    def backward(ctx, grad_output):
        # YOUR CODE HERE
        
        val, =  ...
        grad = ...
        
        # END OF YOUR CODE
        return grad

In [ ]:
from torch.autograd import gradcheck

In [ ]:
sigmoid = MySigmoid.apply
x = torch.rand(2, requires_grad=True)
print(gradcheck(sigmoid, x, eps=1e-4, atol=1e-3))

# be sure to use double for better approximation
x = torch.rand(2, requires_grad=True).double()
print(gradcheck(sigmoid, x, eps=1e-6, atol=1e-4))

PyTorch умеет считать матрицу Якоби или матрицу Гессе для заданной функции.

In [ ]:
from torch.autograd.functional import hessian, jacobian

In [ ]:
jacobian(sigmoid, x)

In [ ]:
def sum_sigmoid(x):
    return torch.sum(sigmoid(x))

In [ ]:
hessian(sum_sigmoid, x)

## Итоги

* Узнали, что такое `pytorch` и как в нем работать с тензорами
* Немного погрели GPU, запустив вычисления на нем
* Решили пример задачи, используя только библиотеку `pytorch`
* Написали собственную реализацию сигмоидальной нелинейности

Что еще почитать:
* [A gentle introduction to `torch.autograd`](https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html)
* [Extending pytorch](https://pytorch.org/docs/stable/notes/extending.html#extending-torch-autograd)

В следующий раз: 
* (наконец-то) нейросети на pytorch